In [45]:
pip install pandas plotly

In [46]:
import pandas as pd

df0 = pd.read_csv('data/0_preprocessed.csv')
df0

,timestamp,duration,song,artist,album,song_artist
0,7/5/2019 3:00,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande
1,7/5/2019 3:01,1.29,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
2,7/5/2019 3:05,3.67,I Don't Care,Ed Sheeran,I Don't Care,I Don't Care - Ed Sheeran
3,7/5/2019 3:08,3.30,Beautiful People,Ed Sheeran,Beautiful People,Beautiful People - Ed Sheeran
4,7/5/2019 3:12,3.32,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
...,...,...,...,...,...,...
47672,9/4/2024 13:36,3.15,The Lazy Song,Bruno Mars,Doo-Wops & Hooligans,The Lazy Song - Bruno Mars
47673,9/4/2024 14:35,2.64,Part Of Me,Katy Perry,Teenage Dream: The Complete Confection,Part Of Me - Katy Perry
47674,9/5/2024 13:50,0.00,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan
47675,9/5/2024 16:08,0.07,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan


In [47]:
df0['timestamp'] = pd.to_datetime(df0['timestamp'])

In [48]:
from datetime import datetime, timedelta

# Define the start and end dates
start_date = datetime(2019, 7, 5)
end_date = datetime(2024, 9, 12)

# Create an empty list to store the timestamps
timestamps = []

# Generate timestamps with a daily increment
current_date = start_date
while current_date <= end_date:
  timestamps.append(current_date)
  current_date += timedelta(weeks=1)

# Print the list of timestamps
print(len(timestamps), timestamps[0], timestamps[-1])

271 2019-07-05 00:00:00 2024-09-06 00:00:00


In [49]:
data_gantt = {
  'timestamp_start': [],
  'timestamp_end': [],
  'most_played_artist': [],
  'most_played_song': [],
  'artist_duration': [],
  'song_duration': []
}

# Iterate over the timestamps
for i in range(len(timestamps) - 1):
  # Get the current timestamp and the next timestamp
  current_timestamp = timestamps[i]
  next_timestamp = timestamps[i+1]

  # Filter the DataFrame based on the timestamps
  filtered_df = df0[(df0['timestamp'] >= current_timestamp) & (df0['timestamp'] < next_timestamp)]

  # Calculate the total duration for each artist
  artist_duration = filtered_df.groupby('artist')['duration'].sum().astype(int)

  # Find the artist with the most played time
  most_played_artist = artist_duration.idxmax() if not artist_duration.empty else None
  most_played_artist_duration = artist_duration.max() if not artist_duration.empty else 0

  # Calculate the total duration for each song
  song_duration = filtered_df.groupby('song')['duration'].sum().astype(int)

  # Find the song with the most played time
  most_played_song = song_duration.idxmax() if not song_duration.empty else None
  most_played_song_duration = song_duration.max() if not song_duration.empty else 0

  # Append the data to the list
  data_gantt['timestamp_start'].append(current_timestamp)
  data_gantt['timestamp_end'].append(next_timestamp)
  data_gantt['most_played_artist'].append(most_played_artist)
  data_gantt['artist_duration'].append(most_played_artist_duration)
  data_gantt['most_played_song'].append(most_played_song)
  data_gantt['song_duration'].append(most_played_song_duration)

# Create a new DataFrame from the data
df_gantt = pd.DataFrame(data_gantt)

# Print the new DataFrame
df_gantt

,timestamp_start,timestamp_end,most_played_artist,most_played_song,artist_duration,song_duration
0,2019-07-05,2019-07-12,Alan Walker,All Falls Down,69,50
1,2019-07-12,2019-07-19,Taylor Swift,Delicate,189,95
2,2019-07-19,2019-07-26,Taylor Swift,Everything Has Changed,653,354
3,2019-07-26,2019-08-02,Taylor Swift,Everything Has Changed,201,127
4,2019-08-02,2019-08-09,Taylor Swift,Everything Has Changed,265,95
...,...,...,...,...,...,...
265,2024-08-02,2024-08-09,Sơn Tùng M-TP,Nuvole Bianche,113,29
266,2024-08-09,2024-08-16,Sơn Tùng M-TP,Nơi Này Có Anh,246,67
267,2024-08-16,2024-08-23,Sơn Tùng M-TP,Nơi Này Có Anh,548,209
268,2024-08-23,2024-08-30,Sơn Tùng M-TP,Chắc Ai Đó Sẽ Về,1460,403


In [50]:
n_artists_otw = len(df_gantt['most_played_artist'].unique())
n_songs_otw = len(df_gantt['most_played_song'].unique())

In [51]:
# Get the top 9 most played artists
top_9_artists = df_gantt['most_played_artist'].value_counts().head(9).index

# Get the top 9 most played songs
top_9_songs = df_gantt['most_played_song'].value_counts().head(9).index

# Replace values not in the top 9 with "13 others" for artists
df_gantt['most_played_artist'] = df_gantt['most_played_artist'].apply(lambda x: f'{n_artists_otw-9} Others' if x not in top_9_artists else x)

# Replace values not in the top 9 with "65 others" for songs
df_gantt['most_played_song'] = df_gantt['most_played_song'].apply(lambda x: f'{n_songs_otw-9} Others' if x not in top_9_songs else x)
df_gantt

,timestamp_start,timestamp_end,most_played_artist,most_played_song,artist_duration,song_duration
0,2019-07-05,2019-07-12,16 Others,61 Others,69,50
1,2019-07-12,2019-07-19,Taylor Swift,61 Others,189,95
2,2019-07-19,2019-07-26,Taylor Swift,61 Others,653,354
3,2019-07-26,2019-08-02,Taylor Swift,61 Others,201,127
4,2019-08-02,2019-08-09,Taylor Swift,61 Others,265,95
...,...,...,...,...,...,...
265,2024-08-02,2024-08-09,Sơn Tùng M-TP,Nuvole Bianche,113,29
266,2024-08-09,2024-08-16,Sơn Tùng M-TP,Nơi Này Có Anh,246,67
267,2024-08-16,2024-08-23,Sơn Tùng M-TP,Nơi Này Có Anh,548,209
268,2024-08-23,2024-08-30,Sơn Tùng M-TP,61 Others,1460,403


In [52]:
gantt_top_artists = df_gantt['most_played_artist'].value_counts()
gantt_top_artists

,count
most_played_artist,
Taylor Swift,143
Ludovico Einaudi,71
16 Others,21
Sơn Tùng M-TP,15
Westlife,7
Joe Hisaishi,3
Lana Del Rey,3
Andiez,3
Justin Bieber,2


In [53]:
gantt_top_songs = df_gantt['most_played_song'].value_counts()
gantt_top_songs

,count
most_played_song,
61 Others,101
Nuvole Bianche,76
Enchanted,41
Sparks Fly,13
Nơi Này Có Anh,13
Ancora,6
As Long As You Love Me,5
Moonlight Sonata,5
The Portrait,5


In [54]:
import plotly.figure_factory as ff
import plotly.express as px

In [55]:
colors = px.colors.qualitative.Bold
colors_songs = {song : color for song, color in zip(df_gantt['most_played_song'].value_counts().index, colors)}
colors_artists = {artist : color for artist, color in zip(df_gantt['most_played_artist'].value_counts().index, colors)}

In [56]:
gantt_songs, gantt_artists = [], []
for i, row in df_gantt.iterrows():
    song = row['most_played_song']
    artist = row['most_played_artist']
    start = row['timestamp_start']
    finish = row['timestamp_end']
    n_weeks_song = gantt_top_songs[song]
    n_weeks_artist = gantt_top_artists[artist]
    task_song = song + f' {n_weeks_song}w'
    task_artist = artist + f' {n_weeks_artist}w'

    gantt_songs.append(
        dict(
            Task=task_song,
            Start=start,
            Finish=finish,
            Resource=song
        )
    )

    gantt_artists.append(
        dict(
            Task=task_artist,
            Start=start,
            Finish=finish,
            Resource=artist
        )
    )

In [57]:
fig_songs = ff.create_gantt(gantt_songs, title='Most-played Song of the Week',colors=colors_songs, index_col='Resource', group_tasks=True, height=500, bar_width=0.5)
fig_songs.show()

In [58]:
fig_artists = ff.create_gantt(gantt_artists, title='Most-played Artist of the Week',colors=colors_artists, index_col='Resource', group_tasks=True, height=500, bar_width=0.5)
fig_artists.show()

In [60]:
df_gantt.to_csv('3_gantt_top_otw.csv', index=False)